In [1]:
from openai import OpenAI

client = OpenAI(
    # #将这里换成你在aiproxy api keys拿到的密钥
    api_key="fk204054-M7aFv2t5pdWOSBMTURsUClih3sPHb8qQ|ck872-e5b3a2d",
    # 这里将官方的接口访问地址，替换成aiproxy的入口地址
    base_url="https://openai.api2d.net/v1"
)

def get_completion(prompt, stream=False, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages = messages,
        model = model,
        stream = stream,
        response_format={ "type": "json_object" },
    )
    # 如果是连续输出模式
    if stream==True:
        # 返回一个生成器
        return response
    else:
        # 一次性输出模式
        return response.choices[0].message.content

In [2]:
import os
from PyPDF2 import PdfReader
import pandas as pd
import json

# 定义PDF文件所在的文件夹
folder_path = "meta"

# 获取文件夹中所有PDF文件的文件名
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

# 初始化一个空的DataFrame来存储所有的文献数据
all_reviews_df = pd.DataFrame(columns=['主要作者和年份', '国家和地区', '干预措施和随访时间', '干预的具体内容',
                                      '样本量', '干预执行者', '技术手段', '对照组'])

# 遍历所有PDF文件
for pdf_file in pdf_files:
    # 创建PDF读取器对象
    pdf_path = os.path.join(folder_path, pdf_file)
    reader = PdfReader(pdf_path)
    
    # 获取PDF的页数
    num_pages = len(reader.pages)
    # 判断页数是否小于50页
    if num_pages < 50:
        # 初始化一个空字符串来存储整个PDF的内容
        pdf_text = ""

        # 遍历PDF的每一页并提取文本
        for page in reader.pages:
            pdf_text += page.extract_text()

        # 调用API获取JSON格式数据
        json_data = get_completion(prompt = f'''我要写一篇关于VR和老年痴呆的meta分析论文，我们需要从下面这篇文献中提取以下信息，如果没有相关信息则填“无”：
    1.文章的主要作者和年份 
    2.研究所在的国家和地区 
    3.干预措施和随访时间 
    4.干预的具体内容 
    5.样本量 
    6.干预执行者 
    7.技术手段 
    8.对照组 

    ========= 
    \n{pdf_text}\n 以json形式输出,例如：
      {{
        "主要作者和年份": "Doniger et al., 2018",
        "国家和地区": "无",
        "干预措施和随访时间": "VR cognitive-motor training, 45 min, twice/week for 12 weeks",
        "干预的具体内容": "无",
        "样本量": "125",
        "干预执行者": "无",
        "技术手段": "VR cognitive-motor training",
        "对照组": "无"
      }}''')

        print(json_data)
        # 将JSON字符串转换为字典
        json_data = json.loads(json_data)

        # 按顺序提取json_data的值
        data_row = list(json_data.values())

        # 确保长度匹配DataFrame的列数（如果json_data的keys和DataFrame的columns一致，此步骤是多余的）
        if len(data_row) == len(all_reviews_df.columns):
            df = pd.DataFrame([data_row], columns=all_reviews_df.columns)
            # 将df追加到all_reviews_df中
            all_reviews_df = pd.concat([all_reviews_df, df], ignore_index=True)
        else:
            print(f"Error: Data length mismatch in file {pdf_file}")
    else:
        # 大于50页的长文档额外处理
        pass

# 输出最终结果
print(all_reviews_df)


{
  "主要作者和年份": "Irina Galperin et al., 2023",
  "国家和地区": "以色列和美国，德国",
  "干预措施和随访时间": "三维虚拟现实结合跑步机训练，6周，3次/周，随后3个月的随访",
  "干预的具体内容": "结合虚拟现实的跑步机训练，测量步态速度和认知处理速度。",
  "样本量": "124",
  "干预执行者": "未明确指出，但在多个临床中心进行",
  "技术手段": "虚拟现实技术，跑步机训练",
  "对照组": "仅跑步机训练组"
}
{
  "主要作者和年份": "Park et al., 2019",
  "国家和地区": "South Korea",
  "干预措施和随访时间": "无",
  "干预的具体内容": "使用虚拟现实技术在精神卫生领域的多种应用，包括针对老年痴呆、轻度认知障碍、焦虑症、恐惧症、精神分裂症和自闭症的治疗",
  "样本量": "无",
  "干预执行者": "无",
  "技术手段": "虚拟现实",
  "对照组": "无"
}
{
  "主要作者和年份": "Liao et al., 2020",
  "国家和地区": "台湾，台北",
  "干预措施和随访时间": "VR-based physical and cognitive training, 60 minutes, three times a week for 12 weeks",
  "干预的具体内容": "每次参与者进行40分钟的VR身体训练和20分钟的认知训练，包括模拟日常活动的VR游戏",
  "样本量": "34",
  "干预执行者": "经验丰富的物理治疗师",
  "技术手段": "Kinect系统和VIVE系统",
  "对照组": "结合身体和认知训练(CPC)组"
}
                       主要作者和年份        国家和地区  \
0  Irina Galperin et al., 2023    以色列和美国，德国   
1            Park et al., 2019  South Korea   
2            Liao et al., 2020        台湾，台北   

                  

In [3]:
all_reviews_df

,主要作者和年份,国家和地区,干预措施和随访时间,干预的具体内容,样本量,干预执行者,技术手段,对照组
0,"Irina Galperin et al., 2023",以色列和美国，德国,三维虚拟现实结合跑步机训练，6周，3次/周，随后3个月的随访,结合虚拟现实的跑步机训练，测量步态速度和认知处理速度。,124,未明确指出，但在多个临床中心进行,虚拟现实技术，跑步机训练,仅跑步机训练组
1,"Park et al., 2019",South Korea,无,使用虚拟现实技术在精神卫生领域的多种应用，包括针对老年痴呆、轻度认知障碍、焦虑症、恐惧症、精...,无,无,虚拟现实,无
2,"Liao et al., 2020",台湾，台北,"VR-based physical and cognitive training, 60 m...",每次参与者进行40分钟的VR身体训练和20分钟的认知训练，包括模拟日常活动的VR游戏,34,经验丰富的物理治疗师,Kinect系统和VIVE系统,结合身体和认知训练(CPC)组


In [5]:
csv_path = '数据提取.csv'
all_reviews_df.to_csv(csv_path, index=False, encoding='utf8' )